In [1]:
import pandas as pd

TRAIN = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'

df = pd.read_csv(filepath_or_buffer=TRAIN,).drop(columns=['Id'])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
TEST = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
test_df = pd.read_csv(filepath_or_buffer=TEST)

For this exercise we are going to use scikit-learn regression models only for data with no missing values.

In [3]:
TARGET = 'SalePrice'
keys = [key for key, value in df.drop(columns=[TARGET]).dtypes.to_dict().items() if str(value) in {'int64', 'float64'}]
numerical_columns = [key for key in keys if test_df[key].isna().sum() == 0]

In [4]:
from plotly import express

express.histogram(x=df[TARGET].tolist())

Fortunately we have no zero or negative values in our training data, but we do have some outliers above about $450k that may cause us some problems.

In [5]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def rmse(y_true, y_pred) -> float:
    return mean_squared_error(squared=False, y_true=np.log(y_true), y_pred=np.log(y_pred))

X_train, X_test, y_train, y_test = train_test_split(df[numerical_columns], df[TARGET], test_size=0.20, random_state=2024)
print('train/test sizes: {}/{}'.format(len(X_train), len(X_test)))

train/test sizes: 1168/292


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor

ACTIVATION = ['identity', 'logistic', 'tanh', 'relu'][3]
CRITERION = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'][3]
MODELS = {
    'linear regression -' :  LinearRegression(positive=False),
    'linear regression +' :  LinearRegression(positive=True),
    'ridge -': Ridge(tol=1e-4, random_state=2024, max_iter=10000, positive=False, ),
    'ridge +': Ridge(tol=1e-4, random_state=2024, max_iter=10000, positive=True, solver='lbfgs'),
    'lasso -': Lasso(tol=1e-4, random_state=2024, max_iter=10000, positive=False, ),
    'lasso +': Lasso(tol=1e-4, random_state=2024, max_iter=10000, positive=True, ),
    'neural network' : MLPRegressor(hidden_layer_sizes=(400, 200, 100,), activation=ACTIVATION, solver='adam', alpha=1e-3, batch_size='auto', 
                      learning_rate='adaptive', learning_rate_init=1e-2, power_t=0.5, max_iter=1000, shuffle=True, random_state=2024,
                      tol=1e-5, verbose=False, warm_start=False, momentum=0.8, nesterovs_momentum=True, early_stopping=False, beta_1=0.9, 
                      beta_2=0.999, epsilon=1e-08, n_iter_no_change=20, max_fun=15000),
    'tree' : DecisionTreeRegressor(criterion=CRITERION, splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0, max_features=None, random_state=2024, max_leaf_nodes=None,
                             min_impurity_decrease=0.0, ccp_alpha=0.0, ),
}

for name, model in MODELS.items():
    model.fit(X=X_train, y=y_train)
    score = rmse(y_true=y_test, y_pred=model.predict(X=X_test))
    print('{:5.4f} {}'.format(score, name))

0.1671 linear regression -
0.1873 linear regression +
0.1670 ridge -
0.1836 ridge +
0.1671 lasso -
0.1873 lasso +
0.1980 neural network
0.2076 tree


In [7]:
from plotly import express
scores = []
model = MODELS['linear regression -']
for random_state in range(200):
    print(random_state)
    X_train, X_test, y_train, y_test = train_test_split(df[numerical_columns], df[TARGET], test_size=0.20, random_state=random_state, )
#     scores.append(
#         rmse(y_true=y_test, y_pred=model.fit(X=X_train, y=y_train).predict(X=X_test))
#     )

# express.line(y=scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
